# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import pandas as pd
pd.options.display.max_columns = None
client = MongoClient()

In [2]:
client.list_database_names()

['admin', 'alojamientos', 'config', 'ironhack', 'local']

In [3]:
'''
En este caso, sigo los pasos de la clase para comenzar a declarar una coleccion
Esto deriva de mydb, la cual es la carpeta "ironhack", donde se encuentra el jason de companies
Seguiré trabajandolo en inglés ya que me ayuda con el nombre de las columnas del Jason
'''

mydb = client['ironhack']
companies = mydb.companies

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
'''
esta vez he impreso el resultado de lista utilizando el diccionario de name. 
para que solo me salga el name, utilizo otro k:v donde name es la key y 1 es value
_id : 0 para que no aparezca el codigo anterior'''

list(companies.find({"name":"Babelgum"},{'name':1, '_id': 0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [5]:
'''
he creado una dataframe en donde le voy a buscar las companies que sean mayor que 5000($gt)
después utilizando el sort(), lo he puesto de orden descendiente con -1
imprimo los primeros 20 con head(21)'''

mayor5k = pd.DataFrame(companies.find({'number_of_employees': {'$gt':5000 }}).sort("number_of_employees", -1))
mayor5k.head(21)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,1847.0,NaN,NaN,None,None,None,None,"automation, building-technologies, drive-techn...",,contact@siemens.com,49 89 636 34134,Electronics and Electrical Engineering,Thu Jul 31 09:29:43 UTC 2008,Thu Nov 28 20:32:55 UTC 2013,"<p>Siemens AG, an electronics and electrical e...","{'available_sizes': [[[150, 34], 'assets/image...",[],"[{'is_past': False, 'title': 'Chairman, Superv...",[],"[{'title': 'Application Development', 'is_past...",$8.9M,"[{'id': 17062, 'round_code': 'grant', 'source_...",[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,1896.0,NaN,NaN,None,None,None,,,,ews@us.ibm.com,914-499-1900,,Fri Mar 14 22:55:52 UTC 2008,Sat Jan 04 02:56:24 UTC 2014,"<p>IBM, acronym for International Business Mac...","{'available_sizes': [[[150, 60], 'assets/image...","[{'name': 'Lotus Notes', 'permalink': 'lotus-n...","[{'is_past': False, 'title': 'Chairman of the ...","[{'competitor': {'name': 'BEA Systems', 'perma...","[{'title': '', 'is_past': False, 'provider': {...",$0,[],"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]
2,52cdef7d4bab8bd675299d33,Toyota,toyota,http://www.crunchbase.com/company/toyota,http://www.toyota-global.com,,,Toyota,enterprise,320000,1933.0,NaN,NaN,None,None,None,None,None,,,81 5 6528 2121,Automotive,Mon Sep 22 22:00:26 UTC 2008,Mon Nov 11 16:24:21 UTC 2013,<p>Toyota operates 75 manufacturing companies ...,"{'available_sizes': [[[135, 37], 'assets/image...",[],"[{'is_past': False, 'title': '', 'person': {'f...",[],"[{'title': '', 'is_past': True, 'provider': {'...",$0,[],"[{'funding_round': {'round_code': 'a', 'source...",None,[],"[{'description': 'HQ', 'address1': 'Toyota Mot...",[],None,"[{'embed_code': '<iframe width=""430"" height=""3...","[{'available_sizes': [[[150, 84], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000,1998.0,12.0,1.0,None,None,None,None,"moneytransfer, crunchbase, ecommerce, paypal, ...",,,207-619-2873,Internet payment service,Wed Aug 01 09:16:46 UTC 2007,Fri Dec 20 04:43:53 UTC 2013,<p>PayPal is an online payments and money tran...,"{'available_sizes': [[[150, 51], 'assets/image...","[{'name': 'PayPal Here', 'permalink': 'paypal-...","[{'is_past': False, 'title': 'Vice President, ...","[{'competitor': {'name': 'KushCash', 'permalin...","[{'title': 'Public Relations', 'is_past': True...",$197M,"[{'id': 284, 'round_code': 'd', 'source_url': ...","[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [6]:
'''
1. he creado las condiciones
    founded_year que sea mayor o igual ($gte) a 2000
    founded_year que sea menor o igual ($lte) a 2500
2. crear la df between, utilizando las condiciones para un find, y utilizando el $and para que me considere ambas
3. lo he sorteado por orden alfabetico por gusto
4. btwdf es una dataframe donde he creado un loc, para imprimir solo 'name' y 'founded_year'
5. imprimir 

'''
cond1 = {'founded_year':{'$gte':2000}}
cond2 = {'founded_year':{'$lte':2005}}

between = pd.DataFrame(companies.find({'$and':[cond1,cond2]}).sort('name',1))
btwdf = between.loc[:,['name','founded_year']]
btwdf.head(5)

,name,founded_year
0,10East,2002
1,1915 Studios,2004
2,1938 Media,2005
3,1Scan,2004
4,2 Minutes,2000


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [7]:
'''
1. he creado las condiciones
    founded_year que sea mayor o igual ($gte) a 2000
    founded_year que sea menor o igual ($lte) a 2500
2. crear la df between, utilizando las condiciones para un find, y utilizando el $and para que me considere ambas
3. lo he sorteado por orden alfabetico por gusto
4. btwdf es una dataframe donde he creado un loc, para imprimir solo 'name' y 'founded_year'
5. imprimir 

'''

#valuation = pd.DataFrame(companies.find({'founded_year':{'$lt':2010}}))

#x = {'valuation_amount': '5', 'valuation_currency': 'None'}
#y = 'None'

#def limpiar_valuacion(y):
#    if valuation['ipo'] != y:
#        valuation['valuation_buena'] = valuation['ipo']['valuation_amount']
#    else:
#        0
#    return valuation

#valuation['valuation_bon'] = valuation['ipo'].apply(limpiar_valuacion)
#valuation.head(2)

#cond_1 = {'ipo':{'$gt':100000000}}
#cond_2 = {'founded_year':{'$lt':2010}}

#valuation = pd.DataFrame(companies.find({'$and':[cond_1,cond_2]}))
#valdf = valuation.loc[:,['name','ipo']]
#valuation.head(5)
#companies.head()

"\n1. he creado las condiciones\n    founded_year que sea mayor o igual ($gte) a 2000\n    founded_year que sea menor o igual ($lte) a 2500\n2. crear la df between, utilizando las condiciones para un find, y utilizando el $and para que me considere ambas\n3. lo he sorteado por orden alfabetico por gusto\n4. btwdf es una dataframe donde he creado un loc, para imprimir solo 'name' y 'founded_year'\n5. imprimir \n\n"

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [28]:
'''
1. he creado las condiciones
    number_of_employees que sea menos ($lt) a 1000
    founded_year que sea menor  ($lt) a 2005
2. crear la b4, utilizando las condiciones para un find, y utilizando el $and para que me considere ambas
3. lo he sorteado por orden de numero de empleados
4. b4_ es una dataframe donde he creado un loc, para imprimir solo 'name' y 'founded_year' y number of employees, por gusto
5. imprimir 

'''
cond1 = {'number_of_employees':{'$lt':1000}}
cond2 = {'founded_year':{'$lt':2005}}

b4 = pd.DataFrame(companies.find({'$and':[cond1,cond2]}).sort('number_of_employees',-1))
b4_ = b4.loc[:,['name','founded_year','number_of_employees']]
b4_.head(10)


,name,founded_year,number_of_employees
0,Infinera Corporation,2000,974
1,NorthPoint Communications Group,1997,948
2,888 Holdings,1997,931
3,Forrester Research,1983,903
4,SonicWALL,1991,900
5,Webmetrics,1999,900
6,Cornerstone OnDemand,1999,881
7,Yelp,2004,800
8,MySpace,2003,800
9,ZoomInfo,2000,800


### 6. All the companies that don't include the `partners` field.

In [32]:
'''
1. crear una dataframe donde los valores en "partners" == [] 
2. imprimir

'''

no_part = pd.DataFrame(companies.find({'partners':[]}))
no_part.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,10.0,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,2007-05-25 06:51:27,Sun Dec 08 07:15:44 UTC 2013,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,NaN,NaN,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,2007-05-25 19:24:22,Wed Oct 31 18:26:09 UTC 2012,"<p>AdventNet is now <a href=""/company/zoho-man...","{'available_sizes': [[[150, 55], 'assets/image...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN


### 7. All the companies that have a null type of value on the `category_code` field.

In [43]:
'''
1. crear una dataframe donde los valores en "category_code" == None
2. imprimir

'''

no_cat = pd.DataFrame(companies.find({'category_code': None}))
no_cat.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd6752980f6,Collective,collective,http://www.crunchbase.com/company/collective,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,Thu Sep 26 13:15:02 UTC 2013,Thu Sep 26 13:15:02 UTC 2013,None,None,[],[],[],[],$0,[],[],None,[],[],[],None,[],[],[],[]
1,52cdef7c4bab8bd675298225,Snimmer,snimmer,http://www.crunchbase.com/company/snimmer,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,Sat Feb 09 20:35:27 UTC 2008,Mon Mar 10 01:30:14 UTC 2008,None,None,[],[],[],[],$0,[],[],None,[],[],[],None,[],[],[],[]


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [46]:
'''
1. he creado las condiciones
    number_of_employees que sea menos ($lt) a 1000
    founded_year que sea menor  ($lt) a 2005
2. crear la b4, utilizando las condiciones para un find, y utilizando el $and para que me considere ambas
3. lo he sorteado por orden de numero de empleados
4. b4_ es una dataframe donde he creado un loc, para imprimir solo 'name' y 'founded_year' y number of employees, por gusto
5. imprimir 

'''
cnd1 = {'number_of_employees':{'$gte':100}}
cnd2 = {'number_of_employees':{'$lte':1000}}

empl= pd.DataFrame(companies.find({'$and':[cnd1,cnd2]}))
empl_ = empl.loc[:,['name','number_of_employees']]
empl_.head(5)

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [12]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [54]:
top10 = pd.DataFrame(companies.find().sort('number_of_employees',-1))
top10_clean = top10.loc[:,['name','number_of_employees']]
top10_clean.head(11)

,name,number_of_employees
0,Siemens,405000.0
1,IBM,388000.0
2,Toyota,320000.0
3,PayPal,300000.0
4,Nippon Telegraph and Telephone Corporation,227000.0
5,Samsung Electronics,221726.0
6,Accenture,205000.0
7,Tata Consultancy Services,200300.0
8,Flextronics International,200000.0
9,Safeway,186000.0


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [14]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [15]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [16]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [17]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [18]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [19]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [20]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [21]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [22]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [23]:
# Your Code